# Basic Classification

This notebook would contain basic data cleaning and classification


In [4]:
import pandas as pd

df_msa = pd.read_csv('data/msa-by-zip.csv')

df_msa = df_msa[df_msa['ZIP CODE'] >= 601]

df_msa['ZIP CODE'] = df_msa['ZIP CODE'].astype(str).str.zfill(5)

df_msa.columns = [col.lower().replace(" ", "_") for col in df_msa.columns]

df_subset = df_msa[['zip_code', 'msa_name']]

def clean_data(df, df_subset):
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    df = df.drop([df.columns[0], df.columns[-1]], axis=1)

    df["zip_code"] = df["Geographic Area Name"].str.extract(r'ZCTA5 (\d{5})')

    df = df.drop(columns='Geographic Area Name')

    df.columns = [col.lower().replace(" ", "_") for col in df.columns]

    df['col_numeric'] = pd.to_numeric(df['estimate!!households!!median_income_(dollars)'], errors='coerce')

    df = df.dropna(subset=['col_numeric'])

    df = df.drop(columns='col_numeric')

    df['estimate!!households!!median_income_(dollars)'] = df['estimate!!households!!median_income_(dollars)'].astype(float)

    df_merged = pd.merge(df, df_subset, on='zip_code', how='left')

    col = ['zip_code', 'msa_name']
    df_merged = df_merged[col + [c for c in df_merged.columns if c not in col]]

    df_merged = df_merged[~df_merged['msa_name'].str.contains('NONMETROPOLITAN', na=False)]

    df_merged = df_merged.dropna(subset=['msa_name'])

    return df_merged

df1 = pd.read_csv('data/2023_acs_income_data.csv')

df1 = clean_data(df1, df_subset)

df2 = pd.read_csv('data/2011_acs_income_data.csv')

df2 = clean_data(df2, df_subset)

df2.head()

# df.to_csv('2023_acs_income_data_cleaned.csv', index=False)

/var/folders/l5/c83q5w7j3wvcv1kvpg3d5qc80000gn/T/ipykernel_38636/1100206439.py:43: DtypeWarning: Columns (2,3,34,35,66,67,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('data/2023_acs_income_data.csv')
/var/folders/l5/c83q5w7j3wvcv1kvpg3d5qc80000gn/T/ipykernel_38636/1100206439.py:47: DtypeWarning: Columns (2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('data/2011_acs_income_data.csv')


KeyError: 'estimate!!households!!median_income_(dollars)'

In [ ]:
df1_specific = df1[df1['msa_name'] == 'Austin-Round Rock, TX MSA']

median_income1 = df1_specific['estimate!!households!!median_income_(dollars)'].median()

median_income1

ValueError: could not convert string to float: '-'